### Retrieve Data
Retrieves all fundamental data by ticker for a given exchange and store. as separate csv files

In [3]:
import operator
import math
import random
import pandas as pd
import datetime as dt
import numpy
import os
import urllib
import json
import requests

from io import StringIO
from pathlib import Path
from functools import partial

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

from eod import EodHistoricalData

pd.set_option('display.max_rows', None)

In [31]:
#paths for data - set prefix to location of Data folder
path_prefix = r'C:\Users\OEM\GDrive\WQU'
path_att = r'\Data\fundamentals_by_attribute'
path_fun = r'\Data\fundamentals_by_ticker'
path_std = r'\Data\standardised_fundamentals'
path_eda = r'\Data\exploratory_data_analysis'

In [14]:
os.chdir(path_prefix)

In [4]:
#instantiate datasource client with api
api_key = "618f834b7bfe27.18132752"
client = EodHistoricalData(api_key)

In [5]:
#resp = client.get_exchanges()

In [15]:
#retrieve list of tickers for New York Stock Exchange
resp = client.get_exchange_symbols(exchange='NYSE')

In [16]:
#filter just ticker symbols which are stocks (ETF and Mutual Funds etc. excluded)
tickers = []
for i in range(len(resp)):
    if resp[i]['Type'] == 'Common Stock' or resp[i]['Type'] == 'Preferred Stock' :
        ticker = resp[i]['Code']
        tickers.append(ticker)

In [25]:
path_fun = '\Data\fundamentals_by_ticker'


In [32]:
path_prefix + path_fun + '\df_fun_{}.csv'.format(ticker)

'C:\\Users\\OEM\\GDrive\\WQU\\Data\\fundamentals_by_ticker\\df_fun_A.csv'

In [34]:
#fundamental data is provided in 3 nested dicts - retrieve data
tickers_not_found = []
for ticker in tickers:
    
    filepath = Path(path_prefix + path_fun + '\df_fun_{}.csv'.format(ticker))
    if os.path.isfile(filepath) == False:
        try:
            resp = client.get_fundamental_equity(ticker)

            if resp['General']['Type'] == 'Common Stock' or resp['General']['Type'] == 'Preferred Stock' :
                df_bal = pd.DataFrame.from_dict(resp['Financials']['Balance_Sheet']['quarterly'])
                df_cf = pd.DataFrame.from_dict(resp['Financials']['Cash_Flow']['quarterly'])
                df_inc = pd.DataFrame.from_dict(resp['Financials']['Income_Statement']['quarterly'])
                df_cf.drop(['date', 'filing_date', 'currency_symbol', 'netIncome'], inplace=True)
                df_inc.drop(['date', 'filing_date', 'currency_symbol', 'netIncome'], inplace=True)
                df_fun = pd.concat([df_bal, df_cf, df_inc])

                filepath.parent.mkdir(parents=True, exist_ok=True)  
                df_fun.to_csv(filepath)

            else:
                print(resp['General']['Type'])

        except KeyError:
            print('KeyError for {}'.format(ticker))
            tickers_not_found.append(ticker)
            
df_not_found = pd.DataFrame(tickers_not_found)
df_not_found.to_csv(path_prefix + path_fun + 'df_not_found')

KeyError for AAIC-P-B
KeyError for AAIC-P-C
KeyError for ACRO-WT
KeyError for ADEX-WT
KeyError for ADVE
KeyError for AEFC
KeyError for AEON-WT
KeyError for AETH
KeyError for AGGH
KeyError for AHLT
KeyError for AIDB
KeyError for ALL-P-J
KeyError for AMAX
KeyError for AMH-P-G
KeyError for AMH-P-H
KeyError for AMPD
KeyError for APCB
KeyError for APIE
KeyError for APMU
KeyError for APOS
KeyError for APUE
KeyError for ASBA
KeyError for ASGI
KeyError for ATH-P-E
KeyError for AUB-P-A
KeyError for AUB-PA
KeyError for AULT-P-D
KeyError for AULT-PD
KeyError for BAC-P-Q
KeyError for BANC-P-F
KeyError for BCAT
KeyError for BDVG
KeyError for BEPH
KeyError for BEPI
KeyError for BETE
KeyError for BETH
KeyError for BFIX
KeyError for BGIG
KeyError for BIGZ
KeyError for BIPI
KeyError for BKKT-WT
KeyError for BMEZ
KeyError for BMTX-WT
KeyError for BNH
KeyError for BNJ
KeyError for BTR
KeyError for BUI
KeyError for BUXX
KeyError for BWNB
KeyError for CARU
KeyError for CCIA
KeyError for CHAT
KeyError for C

ValueError: DataFrame constructor not properly called!